In [ ]:
from tqsdk import TqApi, TqSim, TqAuth, TqKq
import datetime
import time
import pandas as pd
tianqin = TqApi(TqKq(), auth=TqAuth('xxxx', 'xxxxx'))

在使用天勤量化之前，默认您已经知晓并同意以下免责条款，如果不同意请立即停止使用：https://www.shinnytech.com/blog/disclaimer/


2025-10-26 20:46:23 -     INFO - 通知 : 与 wss://free-api.shinnytech.com/t/nfmd/front/mobile 的网络连接已建立
2025-10-26 20:46:23 -     INFO - 通知 x6504368: 与 wss://otg-sim.shinnytech.com/trade 的网络连接已建立
2025-10-26 20:46:23 -     INFO - 通知 x6504368: 登录成功


In [19]:
datetime.datetime.strptime(tianqin.get_quote("SHFE.rb2512").datetime.split('.')[0], '%Y-%m-%d %H:%M:%S')

datetime.datetime(2025, 10, 24, 22, 59, 59)

In [45]:
now = datetime.datetime.now()
now

datetime.datetime(2025, 10, 26, 21, 18, 27, 547339)

In [46]:
now.second

27

In [9]:
def trans_time(ts):
    try:
        from datetime import datetime
        timestamp_s = ts / 1e9
        dt_object = datetime.fromtimestamp(timestamp_s)
        formatted_time = dt_object.strftime("%Y-%m-%d %H:%M:%S")
        return formatted_time
    except:
        return ts

data = tianqin.get_kline_serial('SHFE.rb2512', duration_seconds=60, data_length=10000)
df = data.copy()
df['datetime'] = df['datetime'].apply(trans_time)
df = df[df['datetime']>=df[df['datetime'].str.contains(" 21:00")]['datetime'].iloc[0]]
df['datetime'] = pd.to_datetime(df['datetime'])
df

,datetime,id,open,high,low,close,volume,open_oi,close_oi,symbol,duration
340,2025-09-08 21:00:00,60795.0,3112.0,3112.0,3108.0,3109.0,7.0,7237.0,7239.0,SHFE.rb2512,60
341,2025-09-08 21:01:00,60796.0,3109.0,3109.0,3109.0,3109.0,0.0,7239.0,7239.0,SHFE.rb2512,60
342,2025-09-08 21:02:00,60797.0,3109.0,3109.0,3109.0,3109.0,0.0,7239.0,7239.0,SHFE.rb2512,60
343,2025-09-08 21:03:00,60798.0,3109.0,3109.0,3107.0,3107.0,2.0,7239.0,7239.0,SHFE.rb2512,60
344,2025-09-08 21:04:00,60799.0,3107.0,3107.0,3105.0,3105.0,3.0,7239.0,7239.0,SHFE.rb2512,60
...,...,...,...,...,...,...,...,...,...,...,...
9995,2025-10-24 22:55:00,70450.0,3056.0,3057.0,3056.0,3057.0,2.0,6252.0,6251.0,SHFE.rb2512,60
9996,2025-10-24 22:56:00,70451.0,3057.0,3057.0,3056.0,3056.0,1.0,6251.0,6251.0,SHFE.rb2512,60
9997,2025-10-24 22:57:00,70452.0,3056.0,3056.0,3056.0,3056.0,0.0,6251.0,6251.0,SHFE.rb2512,60
9998,2025-10-24 22:58:00,70453.0,3056.0,3057.0,3056.0,3057.0,1.0,6251.0,6250.0,SHFE.rb2512,60


In [10]:
def aggregate_k_lines(df, date_col='datetime'):
    """
    将分钟行情数据按照夜盘21:00到次日15:00聚合成K线
    """
    # 确保date列是datetime类型
    df = df.copy()
    df.sort_values(date_col, inplace=True)
    df[date_col] = pd.to_datetime(df[date_col])

    # 创建交易日标识
    # 如果时间在21:00-23:59，属于次日交易日
    # 如果时间在00:00-15:00，属于当日交易日
    def get_trading_day(dt):
        if dt.hour >= 21:  # 夜盘时间，属于次日
            return dt.date() + pd.Timedelta(days=1)
        else:  # 白天时间，属于当日
            return dt.date()
    df['trading_day'] = df[date_col].apply(get_trading_day)
    return df

def aggregate_group(group):
    return pd.Series({
        'open': group['open'].iloc[0],
        'high': group['high'].max(),
        'low': group['low'].min(),
        'close': group['close'].iloc[-1],
        'volume': group['volume'].sum(),
        'amount': group.get('amount', group['volume'] * group['close']).sum(),
        'start_time': group['datetime'].iloc[0],
        'end_time': group['datetime'].iloc[-1],
        'bar_count': len(group)
    })

In [20]:
aggregate_k_lines(df).groupby('trading_day').apply(aggregate_group, include_groups=False).reset_index()

,trading_day,open,high,low,close,volume,amount,start_time,end_time,bar_count
0,2025-09-09,3112.0,3123.0,3093.0,3099.0,1051.0,3266559.0,2025-09-08 21:00:00,2025-09-09 14:59:00,345
1,2025-09-10,3092.0,3102.0,3077.0,3090.0,1335.0,4124307.0,2025-09-09 21:00:00,2025-09-10 14:59:00,345
2,2025-09-11,3090.0,3096.0,3068.0,3074.0,1004.0,3094458.0,2025-09-10 21:00:00,2025-09-11 14:59:00,345
3,2025-09-12,3071.0,3114.0,3061.0,3106.0,2427.0,7498121.0,2025-09-11 21:00:00,2025-09-12 14:59:00,345
4,2025-09-13,3112.0,3112.0,3094.0,3094.0,374.0,1159935.0,2025-09-12 21:00:00,2025-09-12 22:59:00,120
5,2025-09-15,3094.0,3116.0,3088.0,3111.0,1189.0,3689754.0,2025-09-15 09:00:00,2025-09-15 14:59:00,225
6,2025-09-16,3116.0,3149.0,3116.0,3142.0,3470.0,10880281.0,2025-09-15 21:00:00,2025-09-16 14:59:00,345
7,2025-09-17,3143.0,3148.0,3119.0,3146.0,2122.0,6649023.0,2025-09-16 21:00:00,2025-09-17 14:59:00,345
8,2025-09-18,3146.0,3154.0,3108.0,3126.0,2832.0,8869428.0,2025-09-17 21:00:00,2025-09-18 14:59:00,345
9,2025-09-19,3126.0,3150.0,3122.0,3147.0,2069.0,6495023.0,2025-09-18 21:00:00,2025-09-19 14:59:00,345


In [1]:
from tianqin_backtrader import MyStore

在使用天勤量化之前，默认您已经知晓并同意以下免责条款，如果不同意请立即停止使用：https://www.shinnytech.com/blog/disclaimer/


In [2]:
s = MyStore('x6504368', 'q6504368')

天勤量化连接中......
2025-10-26 20:21:05 -     INFO - 通知 x6504368: 与 wss://otg-sim.shinnytech.com/trade 的网络连接已建立
2025-10-26 20:21:05 -     INFO - 通知 : 与 wss://free-api.shinnytech.com/t/nfmd/front/mobile 的网络连接已建立
2025-10-26 20:21:05 -     INFO - 通知 x6504368: 登录成功
天勤连接成功......


In [3]:
s.get_daily_data('SHFE.cu2511', 20)

{'trading_day': datetime.date(2025, 9, 17), 'open': 80890.0, 'high': 81000.0, 'low': 80350.0, 'close': 80560.0, 'volume': 40603.0, 'amount': 3272294740.0, 'start_time': Timestamp('2025-09-17 00:50:00'), 'end_time': Timestamp('2025-09-17 14:59:00'), 'bar_count': 235, 'datetime': 739511.6243055556}
{'trading_day': datetime.date(2025, 9, 18), 'open': 79980.0, 'high': 80030.0, 'low': 79430.0, 'close': 79580.0, 'volume': 89310.0, 'amount': 7126145780.0, 'start_time': Timestamp('2025-09-17 21:00:00'), 'end_time': Timestamp('2025-09-18 14:59:00'), 'bar_count': 465, 'datetime': 739512.6243055556}
{'trading_day': datetime.date(2025, 9, 19), 'open': 79680.0, 'high': 80080.0, 'low': 79510.0, 'close': 79850.0, 'volume': 54113.0, 'amount': 4319523660.0, 'start_time': Timestamp('2025-09-18 21:00:00'), 'end_time': Timestamp('2025-09-19 14:59:00'), 'bar_count': 465, 'datetime': 739513.6243055556}
{'trading_day': datetime.date(2025, 9, 20), 'open': 79950.0, 'high': 80190.0, 'low': 79910.0, 'close': 800